# Applied Data Science Capstone

### Segmenting and Clustering Neighborhoods in Toronto

### ***Problem Statement***

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

In [36]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [37]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(url).text

In [38]:
list = BeautifulSoup(source, 'lxml')

In [39]:
table=list.find('table')

In [40]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [41]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data


In [42]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data Cleaning

In [43]:
df=df[df['Borough']!='Not assigned']

In [44]:
df[df['Neighborhood']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [45]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [46]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [47]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [48]:
df_merge.drop_duplicates(inplace=True)

In [49]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [50]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [51]:
df_merge.shape

(103, 3)